# Import Required Packages

In [1]:
from Jupyter import *
import os
from time import sleep
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from getpass import getpass

# Check If folders Exist

In [2]:
MalAnalyst.checkFolder("Samples")
MalAnalyst.checkFolder("Defanged")
empty = MalAnalyst.checkSamples("Samples")
if empty:
    print("No samples provided, need samples or you can download them on the next step...")

Need Samples to continue


# Download Samples From Malware Bazaar?

In [3]:
from Jupyter import * 

answer = input("do you want more samples? ")
if answer == "yes" or answer == "y":
    MalAnalyst.downloadSamplesFromMalBazaar()
else:
    print("Continuing...")

do you want more samples?  y
What Tags would you like to include:  RemcosRat


searching for RemcosRat
ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f.zip has been downloaded
ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f.zip has been extracted...
ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f.zip has been removed...
83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47.zip has been downloaded
83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47.zip has been extracted...
83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47.zip has been removed...
79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835.zip has been downloaded
79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835.zip has been extracted...
79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835.zip has been removed...


# Enumerate Samples

In [4]:
samples=!ls Samples/*
for s in samples:
    print("Samples = " + s)

Samples = Samples/79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835.exe
Samples = Samples/83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47.exe
Samples = Samples/ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f.exe


In [5]:
sampleObj = [MalAnalyst(s) for s in samples]

# Create a subdirectory for the Malware

In [6]:
for obj in sampleObj:
    createSub = MalAnalyst.createSubDirectoryForMalwareSample(obj.sampleName)
    hash = MalAnalyst.retrieveSha256HashSum(obj.sampleName)
    print(createSub)
    print("File hash is: " + hash)
    obj.hash = hash
    obj.hashedPath = createSub

Defanged/79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835
File hash is: 79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835
Defanged/83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47
File hash is: 83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47
Defanged/ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f
File hash is: ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f


# Check Details on Virus-Total

In [7]:
VT_API_KEY = getpass("Enter VirusTotal API Key (leave blank if none)")

Enter VirusTotal API Key (leave blank if none) ········


In [8]:
if VT_API_KEY:
    virustot = VirusTotalPublicApi(VT_API_KEY)
else:
    print("No Virus Total Key Inputted...Skipping")

In [9]:
if VT_API_KEY:
    for obj in sampleObj:
        print("Virus Total info regarding this hashsum: " + obj.hash)
        res = virustot.get_file_report(obj.hash)
        conf = Utils.maliciousConfidence(res)
        print("Confidence = " + str(conf))
        
        if len(sampleObj) >= 5:
            sleep(16)
        
    
else:
    print("No API KEY Skipping...")

Virus Total info regarding this hashsum: 79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835
Confidence = 37.14
Virus Total info regarding this hashsum: 83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47
Confidence = None
Virus Total info regarding this hashsum: ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f
Confidence = 40.85


# Defang Malicious Binary

In [10]:
for obj in sampleObj:
    defangBinary = MalAnalyst.moveAndDefang(obj.sampleName)
    print(defangBinary)
    obj.newSampleName = defangBinary

Defanged/79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835/79fd22e1bb6fa5e88488288e2472fc4323948ae21b5ea4ad0a9692b0ebb8b835.Defanged
Defanged/83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47/83d297c0de9846ebf9db06df45c0b09dd2751bbb2e487d9d6df1128a29da6d47.Defanged
Defanged/ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f/ff4eaf31a3a0f4f99fd9c71caa01388cd50a9917fc16625d549c470cf23cd68f.Defanged


# Create a automatic yara rule

In [ ]:
%%capture
for obj in sampleObj:
    MalAnalyst.retrieveStringsFromYarGen(obj.hashedPath, obj.hash)

In [ ]:
print("YaraGen Rules creation has been completed")

# Pull Strings from Binary

In [ ]:
for obj in sampleObj:
    MalAnalyst.pullStringsEncodedAndUnicode(obj.newSampleName, obj.hashedPath)  #obj.samplename 
    print("Encoded Strings and Unicode files for " + obj.hash + "has been pulled")

# Zip Malware with a password

In [ ]:
for obj in sampleObj:
    MalAnalyst.zipMaliciousSample(obj.newSampleName, obj.hash, obj.hashedPath)
    print(obj.hash + " has been zipped and protected with the password infected")    